# Project Review 3 - Multiple Regression of  CVSS v3 Metrics

### Goals
To predict how accurate **CVSS v3** metrics are at determining vulnerability base scores.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import glob
import random
from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#### First I'm looking at one .json file to determine which which columns are needed.

In [2]:
filename = '../../../ipynb/data/nvd/nvdcve-1.1-2017.json'
df = pd.read_json(filename)
df = json_normalize(df['CVE_Items'])
df.head()

,publishedDate,lastModifiedDate,cve.data_type,cve.data_format,cve.data_version,cve.CVE_data_meta.ID,cve.CVE_data_meta.ASSIGNER,cve.problemtype.problemtype_data,cve.references.reference_data,cve.description.description_data,...,impact.baseMetricV2.cvssV2.availabilityImpact,impact.baseMetricV2.cvssV2.baseScore,impact.baseMetricV2.severity,impact.baseMetricV2.exploitabilityScore,impact.baseMetricV2.impactScore,impact.baseMetricV2.obtainAllPrivilege,impact.baseMetricV2.obtainUserPrivilege,impact.baseMetricV2.obtainOtherPrivilege,impact.baseMetricV2.userInteractionRequired,impact.baseMetricV2.acInsufInfo
0,2017-03-17T00:59Z,2019-10-03T00:03Z,CVE,MITRE,4.0,CVE-2017-0001,cve@mitre.org,"[{'description': [{'lang': 'en', 'value': 'NVD...",[{'url': 'http://www.securityfocus.com/bid/960...,"[{'lang': 'en', 'value': 'The Graphics Device ...",...,COMPLETE,7.2,HIGH,3.9,10.0,False,False,False,False,NaN
1,2017-01-10T21:59Z,2019-10-03T00:03Z,CVE,MITRE,4.0,CVE-2017-0002,cve@mitre.org,"[{'description': [{'lang': 'en', 'value': 'NVD...",[{'url': 'http://www.securityfocus.com/bid/952...,"[{'lang': 'en', 'value': 'Microsoft Edge allow...",...,PARTIAL,6.8,MEDIUM,8.6,6.4,False,False,False,True,NaN
2,2017-01-10T21:59Z,2018-10-12T22:15Z,CVE,MITRE,4.0,CVE-2017-0003,cve@mitre.org,"[{'description': [{'lang': 'en', 'value': 'CWE...",[{'url': 'http://fortiguard.com/advisory/FG-VD...,"[{'lang': 'en', 'value': 'Microsoft Word 2016 ...",...,COMPLETE,9.3,HIGH,8.6,10.0,False,False,False,True,NaN
3,2017-01-10T21:59Z,2018-10-12T22:15Z,CVE,MITRE,4.0,CVE-2017-0004,cve@mitre.org,"[{'description': [{'lang': 'en', 'value': 'CWE...",[{'url': 'http://www.securityfocus.com/bid/953...,"[{'lang': 'en', 'value': 'The Local Security A...",...,COMPLETE,7.8,HIGH,10.0,6.9,False,False,False,False,True
4,2017-03-17T00:59Z,2019-10-03T00:03Z,CVE,MITRE,4.0,CVE-2017-0005,cve@mitre.org,"[{'description': [{'lang': 'en', 'value': 'CWE...",[{'url': 'http://www.securityfocus.com/bid/960...,"[{'lang': 'en', 'value': 'The Graphics Device ...",...,COMPLETE,6.9,MEDIUM,3.4,10.0,False,False,False,False,NaN


In [3]:
for col in df.columns: 
    print(col)

publishedDate
lastModifiedDate
cve.data_type
cve.data_format
cve.data_version
cve.CVE_data_meta.ID
cve.CVE_data_meta.ASSIGNER
cve.problemtype.problemtype_data
cve.references.reference_data
cve.description.description_data
configurations.CVE_data_version
configurations.nodes
impact.baseMetricV3.cvssV3.version
impact.baseMetricV3.cvssV3.vectorString
impact.baseMetricV3.cvssV3.attackVector
impact.baseMetricV3.cvssV3.attackComplexity
impact.baseMetricV3.cvssV3.privilegesRequired
impact.baseMetricV3.cvssV3.userInteraction
impact.baseMetricV3.cvssV3.scope
impact.baseMetricV3.cvssV3.confidentialityImpact
impact.baseMetricV3.cvssV3.integrityImpact
impact.baseMetricV3.cvssV3.availabilityImpact
impact.baseMetricV3.cvssV3.baseScore
impact.baseMetricV3.cvssV3.baseSeverity
impact.baseMetricV3.exploitabilityScore
impact.baseMetricV3.impactScore
impact.baseMetricV2.cvssV2.version
impact.baseMetricV2.cvssV2.vectorString
impact.baseMetricV2.cvssV2.accessVector
impact.baseMetricV2.cvssV2.accessComplexit

#### All columns with impact.baseMetricV3.cvssV3 are needed, except impact.baseMetricV3.cvssV3.baseSeverity.

In [4]:
df = df.iloc[:,[14,15,16,17,18,19,20,21,22]]
df.columns = ['attackVector',
              'attackComplexity',
              'privilegesRequired',
              'userInteraction',
              'scope', 
              'confidentialityImpact',
              'integrityImpact', 
              'availabilityImpact',
              'baseScore']
df.head()

,attackVector,attackComplexity,privilegesRequired,userInteraction,scope,confidentialityImpact,integrityImpact,availabilityImpact,baseScore
0,LOCAL,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,7.8
1,NETWORK,LOW,NONE,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,8.8
2,LOCAL,LOW,NONE,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,7.8
3,NETWORK,LOW,NONE,NONE,UNCHANGED,NONE,NONE,HIGH,7.5
4,LOCAL,HIGH,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,7.0


In [5]:
del df

#### Now we know which columns are needed, we can read in all the data. This function reads in the .json files and returns a single dataframe.

In [6]:
def import_nvd():
    path = '../../../ipynb/data/nvd/'
    all_nvds = glob.glob(path+'*.json')
    l = []
    
    for filename in all_nvds:
        nvd_df = pd.read_json(filename)
        nvd_df = json_normalize(nvd_df['CVE_Items'])
        
        features = nvd_df.iloc[:, [14,15,16,17,18,19,20,21,22]]
        features.columns = ['attackVector', 
                            'attackComplexity', 
                            'privilegesRequired', 
                            'userInteraction', 
                            'scope', 
                            'confidentialityImpact', 
                            'integrityImpact', 
                            'availabilityImpact', 
                            'baseScore']
        features = features.dropna()
        l.append(features)
    
    return pd.concat(l, ignore_index=True, sort=False)

In [7]:
v3_metrics = import_nvd()
v3_metrics.head()

,attackVector,attackComplexity,privilegesRequired,userInteraction,scope,confidentialityImpact,integrityImpact,availabilityImpact,baseScore
0,LOCAL,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,7.8
1,NETWORK,LOW,NONE,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,8.8
2,LOCAL,LOW,NONE,REQUIRED,UNCHANGED,HIGH,HIGH,HIGH,7.8
3,NETWORK,LOW,NONE,NONE,UNCHANGED,NONE,NONE,HIGH,7.5
4,LOCAL,HIGH,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,7.0


#### Drop rows that have a number as integrityImpact metric
* Should only contain: "HIGH", "LOW", or "NONE" 

In [8]:
v3_metrics = v3_metrics.loc[(v3_metrics['integrityImpact'] == 'HIGH') | (v3_metrics['integrityImpact'] == 'LOW') | (v3_metrics['integrityImpact'] == 'NONE')].reset_index(drop=True)

#### Encode the columns and split the dataframe into X (training data) and y (target values).

In [9]:
le = preprocessing.LabelEncoder()

X = v3_metrics.drop('baseScore', axis=1).apply(le.fit_transform)
y = v3_metrics[['baseScore']]

X.shape

(38480, 8)

* 38480 rows / observations
* 8 columns / features

#### Split the data:
* 70% as training data.
* 30% as testing data.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

#### Select a Linear Regression estimator and train it using the training data.

In [11]:
model = LinearRegression();
model.fit(X_train, y_train);

#### Evaluating the accuracy of the model.

In [12]:
y_pred = model.predict(X_test)

# root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# r-squared
r2 = r2_score(y_test, y_pred)

* The **root mean squared error** measures the average distance between the predicted values and the actual values. The smaller the value, the closer the fit.
* The **r-squared** value measures the strength of the relationship between the model and dependent variable. 

In [13]:
print('root mean squared error =', rmse)
print('r-squared =', r2)

root mean squared error = 0.5331723408823822
r-squared = 0.8919226426100392


* The root mean squared error tells us that, with the given metrics, the model can (on average) predict the base score within about plus/minus **0.53 units** of the actual base score.
* About **89.2%** of the data fits the regression model.

#### Let's use the model to make some predictions.

In [14]:
# lets make 15 predictions and see how close the model is
print('Predicted\tActual\t\tDifference')
for i in range(15):
    index = random.randint(0,X.shape[0])
    metrics = [X.iloc[index].values]
    pred = model.predict(metrics)[0][0]
    actual = y.iloc[index].values[0]
    diff = np.absolute(pred - actual)
    
    print('{0:.2f}'.format(pred), '\t\t', actual, '\t\t', '{0:.2f}'.format(diff))

Predicted	Actual		Difference
6.97 		 7.5 		 0.53
6.10 		 6.5 		 0.40
6.42 		 6.1 		 0.32
7.18 		 7.5 		 0.32
8.10 		 7.8 		 0.30
8.52 		 9.1 		 0.58
6.24 		 6.1 		 0.14
9.70 		 9.8 		 0.10
8.51 		 8.1 		 0.41
8.10 		 7.8 		 0.30
6.10 		 6.5 		 0.40
6.42 		 6.1 		 0.32
5.99 		 5.9 		 0.09
7.18 		 7.5 		 0.32
6.97 		 7.5 		 0.53


In [15]:
X_2 = sm.add_constant(X)
model_2 = sm.OLS(y,X_2).fit()

print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:              baseScore   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                 3.781e+04
Date:                Wed, 11 Dec 2019   Prob (F-statistic):               0.00
Time:                        16:50:01   Log-Likelihood:                -31190.
No. Observations:               38480   AIC:                         6.240e+04
Df Residuals:                   38471   BIC:                         6.247e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.95

/Users/seth/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
